In [2]:
import collections
import pathlib
import re
import string

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import tensorflow_datasets as tfds

In [1]:
import tensorflow_text as tf_text

In [3]:
data_url = 'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz'
dataset = utils.get_file(
    'stack_overflow_16k.tar.gz',
    data_url,
    untar=True,
    cache_dir='stack_overflow',
    cache_subdir='')
dataset_dir = pathlib.Path(dataset).parent

6053888/6053168 [==============================] - 1s 0us/step


In [4]:
list(dataset_dir.iterdir())

[WindowsPath('/tmp/.keras/README.md'),
 WindowsPath('/tmp/.keras/stack_overflow_16k.tar.gz.tar.gz'),
 WindowsPath('/tmp/.keras/test'),
 WindowsPath('/tmp/.keras/train')]

In [10]:
train_dir = dataset_dir/'train'
list(train_dir.iterdir())

[WindowsPath('/tmp/.keras/train/csharp'),
 WindowsPath('/tmp/.keras/train/java'),
 WindowsPath('/tmp/.keras/train/javascript'),
 WindowsPath('/tmp/.keras/train/python')]

In [12]:
sample_file = train_dir/'python/1755.txt'
with open(sample_file) as f:
    print(f.read())

why does this blank program print true x=true.def stupid():.    x=false.stupid().print x



In [13]:
batch_size=32
seed = 42

raw_train_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size = batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [14]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(10):
        print("Question: ", text_batch.numpy()[i][:100], '...')
        print("Label: ", label_batch.numpy()[i])

Question:  b'"my tester is going to the wrong constructor i am new to programming so if i ask a question that can' ...
Label:  1
Question:  b'"blank code slow skin detection this code changes the color space to lab and using a threshold finds' ...
Label:  3
Question:  b'"option and validation in blank i want to add a new option on my system where i want to add two text' ...
Label:  1
Question:  b'"exception: dynamic sql generation for the updatecommand is not supported against a selectcommand th' ...
Label:  0
Question:  b'"parameter with question mark and super in blank, i\'ve come across a method that is formatted like t' ...
Label:  1
Question:  b'call two objects wsdl the first time i got a very strange wsdl. ..i would like to call the object (i' ...
Label:  0
Question:  b'how to correctly make the icon for systemtray in blank using icon sizes of any dimension for systemt' ...
Label:  0
Question:  b'"is there a way to check a variable that exists in a different script than the orig

In [15]:
for i, label in enumerate(raw_train_ds.class_names):
    print('Label', i, 'corresponds to ', label)

Label 0 corresponds to  csharp
Label 1 corresponds to  java
Label 2 corresponds to  javascript
Label 3 corresponds to  python


In [16]:
raw_val_ds = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size = batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [17]:
test_dir = dataset_dir/'test'
raw_test_ds = preprocessing.text_dataset_from_directory(
    test_dir, batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [18]:
VOCAB_SIZE=10000

binary_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='binary')

In [19]:
MAX_SEQUENCE_LENGTH = 250
int_vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

In [20]:
train_text = raw_train_ds.map(lambda text, labels: text)
binary_vectorize_layer.adapt(train_text)
int_vectorize_layer.adapt(train_text)

In [21]:
def binary_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return binary_vectorize_layer(text), label

In [22]:
def int_vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return int_vectorize_layer(text), label

In [23]:
text_batch, label_batch = next(iter(raw_train_ds))
first_question, first_label = text_batch[0], label_batch[0]
print('Question', first_question)
print('Label', first_label)

Question tf.Tensor(b'"function expected error in blank for dynamically created check box when it is clicked i want to grab the attribute value.it is working in ie 8,9,10 but not working in ie 11,chrome shows function expected error..&lt;input type=checkbox checked=\'checked\' id=\'symptomfailurecodeid\' tabindex=\'54\' style=\'cursor:pointer;\' onclick=chkclickevt(this);  failurecodeid=""1"" &gt;...function chkclickevt(obj) { .    alert(obj.attributes(""failurecodeid""));.}"\n', shape=(), dtype=string)
Label tf.Tensor(2, shape=(), dtype=int32)


In [24]:
print("'binary' vectorized question:", 
      binary_vectorize_text(first_question, first_label)[0])

'binary' vectorized question: tf.Tensor([[1. 1. 1. ... 0. 0. 0.]], shape=(1, 10000), dtype=float32)


In [25]:
print("'int' vectorized question:",
      int_vectorize_text(first_question, first_label)[0])

'int' vectorized question: tf.Tensor(
[[  38  450   65    7   16   12  892  265  186  451   44   11    6  685
     3   46    4 2062    2  485    1    6  158    7  479    1   26   20
   158    7  479    1  502   38  450    1 1767 1763    1    1    1    1
     1    1    1    1    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0

In [26]:
print('1289 --->', int_vectorize_layer.get_vocabulary()[1289])
print('313 --->', int_vectorize_layer.get_vocabulary()[313])
print("Vocabulary size: {}".format(len(int_vectorize_layer.get_vocabulary())))

1289 ---> roman
313 ---> source
Vocabulary size: 10000


In [30]:
binary_train_ds = raw_train_ds.map(binary_vectorize_text)
binary_val_ds = raw_val_ds.map(binary_vectorize_text)
binary_test_ds = raw_test_ds.map(binary_vectorize_text)

int_train_ds = raw_train_ds.map(int_vectorize_text)
int_val_ds = raw_val_ds.map(int_vectorize_text)
int_test_ds = raw_val_ds.map(int_vectorize_text)

In [31]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

def configure_dataset(dataset):
    return dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [33]:
binary_train_ds = configure_dataset(binary_train_ds)
binary_val_ds = configure_dataset(binary_val_ds)
binary_test_ds = configure_dataset(binary_test_ds)

int_train_ds = configure_dataset(int_train_ds)
int_val_ds = configure_dataset(int_val_ds)
int_test_ds = configure_dataset(int_test_ds)

In [34]:
binary_model = tf.keras.Sequential([layers.Dense(4)])
binary_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = binary_model.fit(
    binary_train_ds, validation_data=binary_val_ds, epochs=10)

Epoch 1/10
200/200 [==============================] - 10s 46ms/step - loss: 1.2433 - accuracy: 0.5307 - val_loss: 0.9124 - val_accuracy: 0.7831
Epoch 2/10
200/200 [==============================] - 0s 2ms/step - loss: 0.8188 - accuracy: 0.8237 - val_loss: 0.7486 - val_accuracy: 0.8025
Epoch 3/10
200/200 [==============================] - 0s 2ms/step - loss: 0.6508 - accuracy: 0.8624 - val_loss: 0.6633 - val_accuracy: 0.8169
Epoch 4/10
200/200 [==============================] - 0s 2ms/step - loss: 0.5512 - accuracy: 0.8868 - val_loss: 0.6100 - val_accuracy: 0.8269
Epoch 5/10
200/200 [==============================] - 0s 1ms/step - loss: 0.4821 - accuracy: 0.9036 - val_loss: 0.5734 - val_accuracy: 0.8356
Epoch 6/10
200/200 [==============================] - 0s 2ms/step - loss: 0.4300 - accuracy: 0.9183 - val_loss: 0.5468 - val_accuracy: 0.8413
Epoch 7/10
200/200 [==============================] - 0s 2ms/step - loss: 0.3886 - accuracy: 0.9273 - val_loss: 0.5267 - val_accuracy: 0.8413
Epoc

In [40]:
def create_model(vocab_size, num_labels):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, 64, mask_zero=True),
        layers.Conv1D(64, 5, padding='valid', activation='relu', strides=2),
        layers.GlobalMaxPooling1D(),
        layers.Dense(num_labels)
    ])
    return model

In [41]:
int_model = create_model(vocab_size = VOCAB_SIZE+1, num_labels=4)
int_model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])
history = int_model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)

Epoch 1/5
200/200 [==============================] - 5s 23ms/step - loss: 1.3050 - accuracy: 0.3977 - val_loss: 0.7655 - val_accuracy: 0.6956
Epoch 2/5
200/200 [==============================] - 2s 10ms/step - loss: 0.7019 - accuracy: 0.7132 - val_loss: 0.5591 - val_accuracy: 0.7862
Epoch 3/5
200/200 [==============================] - 2s 10ms/step - loss: 0.4397 - accuracy: 0.8519 - val_loss: 0.4961 - val_accuracy: 0.8050
Epoch 4/5
200/200 [==============================] - 2s 10ms/step - loss: 0.2503 - accuracy: 0.9375 - val_loss: 0.4941 - val_accuracy: 0.8100
Epoch 5/5
200/200 [==============================] - 2s 10ms/step - loss: 0.1280 - accuracy: 0.9774 - val_loss: 0.5162 - val_accuracy: 0.8119
